Author: Lourde Hajjar

This notebook applies a CNN to the original (raw data) image dataset, initially testing a basic model without specifying a learning rate. The model was then enhanced by incorporating data augmentation (e.g., rotation, shifting, zooming, and flipping) to improve generalization and batch normalization layers to stabilize training.

The model’s performance was first evaluated using a test set and 10-fold cross-validation, achieving an accuracy of 53%. To further improve performance, a learning rate was added, resulting in a significant accuracy increase to 74%.

# CNN With Augmentation

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Define dataset paths 
data_dir =r"C:\Users\lourd\OneDrive\Desktop\coursework\datasets\3_image\processed\3_og_b"


images = []
labels = []


# Load images 
for label in os.listdir(data_dir):
    label_dir = os.path.join(data_dir, label)
    if os.path.isdir(label_dir):
        for img_file in os.listdir(label_dir):
            img_path = os.path.join(label_dir, img_file)
            img = load_img(img_path, color_mode="grayscale")
            img = load_img(img_path, target_size=(256, 256), color_mode="grayscale")
            img_array = img_to_array(img) / 255.0  
            images.append(img_array)
            labels.append(label)


images = np.array(images)
labels = np.array(labels)

class_names = ['normal', 'malignant', 'benign']


images = images.reshape(-1, 256, 256, 1)


label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(images, encoded_labels, test_size=0.2, random_state=42)


## Define CNN Model

In [ ]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)),
    tf.keras.layers.BatchNormalization(),  
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),  
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),  
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  
])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      320       
                                                                 
 batch_normalization (BatchN  (None, 254, 254, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 batch_normalization_1 (Batc  (None, 125, 125, 64)     256       
 hNormalization)                                                 
                                                        

## Apply Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation 
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)


datagen.fit(x_train)


## Train the Model

In [ ]:
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=16),  
    validation_data=(x_test, y_test),
    epochs=30,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
)


Epoch 1/30
85/85 [==============================] - 32s 366ms/step - loss: 5.5743 - accuracy: 0.4079 - val_loss: 104.3528 - val_accuracy: 0.3531
Epoch 2/30
85/85 [==============================] - 30s 353ms/step - loss: 1.0797 - accuracy: 0.4094 - val_loss: 155.4345 - val_accuracy: 0.3531
Epoch 3/30
85/85 [==============================] - 31s 360ms/step - loss: 1.0576 - accuracy: 0.4094 - val_loss: 86.4029 - val_accuracy: 0.3531
Epoch 4/30
85/85 [==============================] - 29s 341ms/step - loss: 1.0215 - accuracy: 0.4480 - val_loss: 24.2094 - val_accuracy: 0.3531
Epoch 5/30
85/85 [==============================] - 29s 345ms/step - loss: 0.9919 - accuracy: 0.4473 - val_loss: 4.4259 - val_accuracy: 0.3976
Epoch 6/30
85/85 [==============================] - 30s 353ms/step - loss: 0.9988 - accuracy: 0.4547 - val_loss: 0.9208 - val_accuracy: 0.5015
Epoch 7/30
85/85 [==============================] - 29s 346ms/step - loss: 1.0134 - accuracy: 0.4621 - val_loss: 0.8800 - val_accuracy: 

## Evaluate the Model

In [8]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=0)
print('\nTest accuracy:', test_acc)


Test accuracy: 0.5341246128082275


## Print Classification Report

In [ ]:
from sklearn.metrics import f1_score, classification_report
import numpy as np

# Make Predictions on the Test Set

y_pred_prob = model.predict(x_test)

# Convert probabilities to class predictions
y_pred = np.argmax(y_pred_prob, axis=1)


if len(y_test.shape) > 1 and y_test.shape[1] > 1:
    
    y_true = np.argmax(y_test, axis=1)
else:
    
    y_true = y_test

# Calculate F1 Score
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

# Print Results
print("Test Accuracy:", np.mean(y_true == y_pred))
print("F1 Score (Macro):", f1_macro)
print("F1 Score (Weighted):", f1_weighted)

# Print Detailed Classification Report
print("\nClassification Report:")
print(classification_report(y_true, y_pred))


11/11 [==============================] - 2s 182ms/step
Test Accuracy: 0.5341246290801187
F1 Score (Macro): 0.4930347129184017
F1 Score (Weighted): 0.49264818677764777

Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.11      0.19       118
           1       0.83      0.72      0.77       119
           2       0.38      0.81      0.52       100

    accuracy                           0.53       337
   macro avg       0.62      0.55      0.49       337
weighted avg       0.63      0.53      0.49       337



## Print 10-Fold Cross-Validation Results

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import numpy as np

# Initialize KFold cross-validation with 10 splits
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Lists to store metrics for each fold
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
roc_auc_scores = []
true_positive_rates = []
false_positive_rates = []

# Loop over each fold
for train_index, val_index in kf.split(x_train):
   
    x_val_fold = x_train[val_index]
    y_val_fold = y_train[val_index]

    # Predict on the validation set
    y_val_pred_prob = model.predict(x_val_fold)
    y_val_pred = np.argmax(y_val_pred_prob, axis=1)

    # Calculate and store metrics
    accuracy = accuracy_score(y_val_fold, y_val_pred)
    precision = precision_score(y_val_fold, y_val_pred, average='macro')
    recall = recall_score(y_val_fold, y_val_pred, average='macro')
    f1 = f1_score(y_val_fold, y_val_pred, average='macro')

    
    if len(np.unique(y_val_fold)) == 2:
        roc_auc = roc_auc_score(y_val_fold, y_val_pred_prob[:, 1])
        roc_auc_scores.append(roc_auc)

    # Calculate TP and FP rates from confusion matrix
    cm = confusion_matrix(y_val_fold, y_val_pred)
    tp_rate = cm[1, 1] / (cm[1, 1] + cm[1, 0]) if (cm[1, 1] + cm[1, 0]) > 0 else 0
    fp_rate = cm[0, 1] / (cm[0, 1] + cm[0, 0]) if (cm[0, 1] + cm[0, 0]) > 0 else 0

    true_positive_rates.append(tp_rate)
    false_positive_rates.append(fp_rate)
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

# Calculate the average of each metric across all folds
print("10-Fold Cross-Validation Results:")
print("Average Accuracy:", np.mean(accuracy_scores))
print("Average Precision:", np.mean(precision_scores))
print("Average Recall:", np.mean(recall_scores))
print("Average F1 Score:", np.mean(f1_scores))
print("Average TP Rate:", np.mean(true_positive_rates))
print("Average FP Rate:", np.mean(false_positive_rates))

if roc_auc_scores:
    print("Average ROC AUC Score:", np.mean(roc_auc_scores))


5/5 [==============================] - 1s 133ms/step
10-Fold Cross-Validation Results:
Average Accuracy: 0.5370867882808181
Average Precision: 0.60297765643291
Average Recall: 0.5337559122127611
Average F1 Score: 0.4909344507097996
Average TP Rate: 1.0
Average FP Rate: 0.24310606060606058


# CNN With Augmentation and Learning Rate

## Define the Model

In [ ]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)),
    tf.keras.layers.BatchNormalization(),  
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),  
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),  
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 254, 254, 32)      320       
                                                                 
 batch_normalization_3 (Batc  (None, 254, 254, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 127, 127, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 batch_normalization_4 (Batc  (None, 125, 125, 64)     256       
 hNormalization)                                                 
                                                      

## Apply Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Apply Data augmentation 
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)


datagen.fit(x_train)


## Train the model

In [ ]:
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=16),  
    validation_data=(x_test, y_test),
    epochs=30,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
)


Epoch 1/30
85/85 [==============================] - 28s 315ms/step - loss: 2.2296 - accuracy: 0.4889 - val_loss: 9.0225 - val_accuracy: 0.2967
Epoch 2/30
85/85 [==============================] - 25s 299ms/step - loss: 0.8015 - accuracy: 0.6114 - val_loss: 13.1406 - val_accuracy: 0.3976
Epoch 3/30
85/85 [==============================] - 25s 294ms/step - loss: 0.6858 - accuracy: 0.6724 - val_loss: 17.2236 - val_accuracy: 0.4273
Epoch 4/30
85/85 [==============================] - 25s 294ms/step - loss: 0.5728 - accuracy: 0.7043 - val_loss: 11.3045 - val_accuracy: 0.4184
Epoch 5/30
85/85 [==============================] - 64s 762ms/step - loss: 0.5472 - accuracy: 0.7444 - val_loss: 4.7620 - val_accuracy: 0.3561
Epoch 6/30
85/85 [==============================] - 25s 298ms/step - loss: 0.5113 - accuracy: 0.7519 - val_loss: 2.4544 - val_accuracy: 0.4214
Epoch 7/30
85/85 [==============================] - 25s 295ms/step - loss: 0.4818 - accuracy: 0.7689 - val_loss: 3.2147 - val_accuracy: 0.3

## Evaluate the Model

In [14]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=0)
print('\nTest accuracy:', test_acc)


Test accuracy: 0.7448071241378784


## Print the Classification Report

In [ ]:
from sklearn.metrics import f1_score, classification_report
import numpy as np

# Make Predictions on the Test Set

y_pred_prob = model.predict(x_test)

# Convert probabilities to class predictions
y_pred = np.argmax(y_pred_prob, axis=1)


if len(y_test.shape) > 1 and y_test.shape[1] > 1:
   
    y_true = np.argmax(y_test, axis=1)
else:
    
    y_true = y_test

# Calculate F1 Score
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

# Print Results
print("Test Accuracy:", np.mean(y_true == y_pred))
print("F1 Score (Macro):", f1_macro)
print("F1 Score (Weighted):", f1_weighted)

# Print Detailed Classification Report
print("\nClassification Report:")
print(classification_report(y_true, y_pred))


11/11 [==============================] - 2s 167ms/step
Test Accuracy: 0.744807121661721
F1 Score (Macro): 0.7330907798788777
F1 Score (Weighted): 0.7362411691636712

Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.42      0.54       118
           1       1.00      0.95      0.97       119
           2       0.56      0.88      0.69       100

    accuracy                           0.74       337
   macro avg       0.77      0.75      0.73       337
weighted avg       0.78      0.74      0.74       337



## Print the 10-Fold Cross-Validation Results:

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import numpy as np

# Initialize KFold cross-validation with 10 splits
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Lists to store metrics for each fold
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
roc_auc_scores = []
true_positive_rates = []
false_positive_rates = []

# Loop over each fold
for train_index, val_index in kf.split(x_train):
   
    x_val_fold = x_train[val_index]
    y_val_fold = y_train[val_index]

    # Predict on the validation set
    y_val_pred_prob = model.predict(x_val_fold)
    y_val_pred = np.argmax(y_val_pred_prob, axis=1)

    # Calculate and store metrics
    accuracy = accuracy_score(y_val_fold, y_val_pred)
    precision = precision_score(y_val_fold, y_val_pred, average='macro')
    recall = recall_score(y_val_fold, y_val_pred, average='macro')
    f1 = f1_score(y_val_fold, y_val_pred, average='macro')

    
    if len(np.unique(y_val_fold)) == 2:
        roc_auc = roc_auc_score(y_val_fold, y_val_pred_prob[:, 1])
        roc_auc_scores.append(roc_auc)

    # Calculate TP and FP rates from confusion matrix
    cm = confusion_matrix(y_val_fold, y_val_pred)
    tp_rate = cm[1, 1] / (cm[1, 1] + cm[1, 0]) if (cm[1, 1] + cm[1, 0]) > 0 else 0
    fp_rate = cm[0, 1] / (cm[0, 1] + cm[0, 0]) if (cm[0, 1] + cm[0, 0]) > 0 else 0

    true_positive_rates.append(tp_rate)
    false_positive_rates.append(fp_rate)
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

# Calculate the average of each metric across all folds
print("10-Fold Cross-Validation Results:")
print("Average Accuracy:", np.mean(accuracy_scores))
print("Average Precision:", np.mean(precision_scores))
print("Average Recall:", np.mean(recall_scores))
print("Average F1 Score:", np.mean(f1_scores))
print("Average TP Rate:", np.mean(true_positive_rates))
print("Average FP Rate:", np.mean(false_positive_rates))

if roc_auc_scores:
    print("Average ROC AUC Score:", np.mean(roc_auc_scores))


5/5 [==============================] - 1s 120ms/step
10-Fold Cross-Validation Results:
Average Accuracy: 0.709502487562189
Average Precision: 0.7241611186070271
Average Recall: 0.7078537355792155
Average F1 Score: 0.7041295895327543
Average TP Rate: 0.9347744783118651
Average FP Rate: 0.0
